In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
from nltk.corpus import brown
from  gensim.utils import simple_preprocess
from tqdm import tqdm
import random
from dataset import Word2VecSentenceDataset



In [3]:
brown_words = brown.words()
brown_text = " ".join(brown.words()).lower()

In [3]:
num_sents = len(brown.sents())
print("number of sentences:", num_sents)

number of sentences: 57340


In [4]:
len(np.unique(simple_preprocess(brown_text.lower())))

41239

In [5]:
a = {"x":4, "y":3}
list(a.keys())

['x', 'y']

In [5]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecModel, self).__init__()
        self.target_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)

        # Initialize weights
        self.target_embeddings.weight.data.uniform_(-1, 1)
        self.context_embeddings.weight.data.uniform_(-1, 1)

    def forward(self, target_words, context_words, negative_words):
        target_embeds = self.target_embeddings(target_words)
        context_embeds = self.context_embeddings(context_words)
        negative_embeds = self.context_embeddings(negative_words)
 

        positive_score = torch.sum(target_embeds * context_embeds, dim=1)
        negative_score = torch.bmm(negative_embeds, target_embeds.unsqueeze(2)).squeeze()
   
        return -torch.mean(torch.log(torch.sigmoid(positive_score)) + torch.sum(torch.log(torch.sigmoid(-negative_score)), dim=1))


In [6]:
def preprocess_brown_corpus():
    processed_sentences = []
    for sentence in brown.sents():
        processed_sentence = simple_preprocess(' '.join(sentence), deacc=True)  
        processed_sentences.append(processed_sentence)
    return processed_sentences

In [7]:
corpus = preprocess_brown_corpus()

In [14]:

dataset = Word2VecSentenceDataset(corpus, window_size=2, negative_samples=5, total_negative_samples=10000)

In [10]:
print(len(dataset.tokens))
dataset.tokens[:10]

981039


['the',
 'fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'an',
 'investigation',
 'of']

In [11]:
print(len(dataset.data))
dataset.data[:10]

3585262


[(26233, 27914, [31431, 233, 808, 24462, 84]),
 (26233, 36134, [541, 1670, 2709, 272, 133]),
 (27914, 26233, [21181, 189, 13242, 17735, 248]),
 (27914, 36134, [7817, 2524, 17259, 6651, 295]),
 (27914, 1268, [6283, 18, 791, 9, 2658]),
 (36134, 26233, [15651, 163, 26752, 382, 6714]),
 (36134, 27914, [77, 24434, 6909, 39571, 4480]),
 (36134, 1268, [3856, 28778, 9328, 1517, 6790]),
 (36134, 23855, [1855, 35741, 2812, 0, 47]),
 (1268, 27914, [23, 6248, 38490, 4090, 357])]

In [16]:
def prepare_dataset(batch_size=512, window_size=5, negative_samples=5, total_negative_samples=10000):
    # Example corpus
    corpus = preprocess_brown_corpus()
    print("preprocessed")
    # Hyperparameters
    

    # Prepare dataset and dataloader
    dataset = Word2VecSentenceDataset(corpus, window_size, negative_samples, total_negative_samples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print("model loaded")
    return dataset, dataloader


In [17]:

def train(dataset, dataloader, epochs = 10, embedding_dim = 20):

     
     learning_rate = 0.01

     vocab_size = len(dataset.vocab)
     model = Word2VecModel(vocab_size, embedding_dim)
     
     optimizer = optim.SparseAdam(model.parameters(), lr=learning_rate)



     # Example training loop
     for epoch in range(epochs):
          total_loss = 0
          for target, context, negatives in tqdm(dataloader):  
               model.zero_grad()
               loss = model(target, context, negatives)
               loss.backward()
               optimizer.step()
               total_loss += loss.item()
               print(loss.item())
          
          print(f"Epoch {epoch}, Loss: {total_loss}")

     return model


In [18]:

dataset, dataloader = prepare_dataset(batch_size=512, window_size=5, negative_samples=5, total_negative_samples=10000)


preprocessed
model loaded


In [19]:
model = train(dataset, dataloader, epochs=5, embedding_dim = 30)

  0%|          | 2/15925 [00:01<2:03:04,  2.16it/s]

6.118812084197998
6.028980731964111
6.191402912139893
6.031180381774902
5.911285400390625
6.005388259887695
6.007167816162109


  0%|          | 13/15925 [00:01<14:53, 17.82it/s] 

6.042276382446289
5.818857669830322
5.85860013961792
5.959985256195068
5.985980033874512
5.8998847007751465
5.8394880294799805
5.743626117706299
6.018033981323242
5.513816833496094
5.858908653259277


  0%|          | 25/15925 [00:01<08:14, 32.16it/s]

...

  1%|          | 164/15925 [00:05<08:22, 31.35it/s]

4.140848636627197
4.013000011444092
3.8698179721832275
3.8773651123046875
4.129438400268555
3.835348606109619
4.016602516174316
3.9354169368743896
3.776674270629883


  1%|          | 172/15925 [00:05<09:01, 29.09it/s]

4.013361930847168
3.954721450805664
3.7807536125183105
3.781461715698242
3.593843460083008


KeyboardInterrupt: 

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
    with record_function("model_inference"):
        model = train(dataset, dataloader)
print("xxxxxxxxxx")
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


In [ ]:
import cProfile
import pstats

# Create a Profile object
profiler = cProfile.Profile()
profiler.enable()

# The code you want to profile
prepare_dataset()

profiler.disable()

# Create Stats object
stats = pstats.Stats(profiler).sort_stats('time')

# Print the statistics
stats.print_stats()




In [14]:
prof.key_averages().table(sort_by="cpu_time_total", row_limit=10)

''

In [12]:
# Inspect embeddings
word_embeddings = model.target_embeddings.weight.data
print(word_embeddings[dataset.vocab["atlanta"]])

tensor([ 0.8798, -0.0981, -0.0641, -0.6053, -0.0116,  0.1742,  0.4074, -0.1351,
        -0.3801,  0.5048, -1.6327,  0.0111, -0.0943, -0.1630,  1.1513,  0.1729,
        -0.1996, -0.7478, -1.0341,  0.5263,  0.4483,  0.1727,  1.0539, -0.1429,
        -0.2299,  0.7460,  0.1045,  0.5899, -0.3256, -0.1430])


In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(word, word_to_int, int_to_word, embeddings, top_n=5):
    # Get the embedding for the given word
    word_idx = word_to_int[word]
    word_embedding = embeddings[word_idx].reshape(1, -1)
    
    # Calculate cosine similarity between this word and all other words in the vocabulary
    similarities = []
    for i in range(len(embeddings)):
        other_word_embedding = embeddings[i].reshape(1, -1)
        similarity = cosine_similarity(word_embedding, other_word_embedding)[0][0]
        similarities.append((i, similarity))
    
    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Convert indices back to words and filter out the input word
    similar_words = [(int_to_word[sim[0]], sim[1]) for sim in similarities if sim[0] != word_idx]
    
    # Return the top N most similar words, excluding the word itself
    return similar_words[:top_n]




In [14]:
# Example usage
word = 'book'  # The word you want to find similar words for
similar_words = find_most_similar(word, dataset.vocab, dataset.index_to_word, word_embeddings, top_n=5)
print(f"Most similar words to '{word}':", similar_words)

Most similar words to 'book': [('comedie', 0.8072341), ('trempler', 0.8050022), ('often', 0.80381423), ('expe', 0.8015127), ('hors', 0.80094683)]


In [15]:
# Example usage
word = 'twenty'  # The word you want to find similar words for
similar_words = find_most_similar(word, dataset.vocab, dataset.index_to_word, word_embeddings, top_n=5)
print(f"Most similar words to '{word}':", similar_words)

Most similar words to 'twenty': [('thirty', 0.8679474), ('ten', 0.8436272), ('six', 0.8391596), ('laid', 0.8352573), ('five', 0.82961893)]
